In [1]:
import numpy as np
import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

# Load a ground truth heightmap

From https://lunar-autonomy-challenge.jhuapl.edu/Challenge-Documentation/index.php#introduction:

- Environments are 40m x 40m in size, with a mapping area of 27m x 27m
- The mapping area is divided up into cells of 15cm x 15cm, hence 180 x 180 cells, so total 32,400 cells


In [ ]:
heightmap_path = "../../data/heightmaps/Moon_Map_01_0_rep0.dat"
heightmap_points = np.load(heightmap_path, allow_pickle=True)
heightmap_points.shape

The heightmap is of shape 180 x 180 x 4, where the 4 channels are (x, y, z, and rock presence (0 or 1))


In [ ]:
fig = go.Figure(
    data=[
        go.Surface(
            x=heightmap_points[:, :, 0], y=heightmap_points[:, :, 1], z=heightmap_points[:, :, 2]
        )
    ]
)
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# Wheel contact height mapping


In [28]:
import json

from lac.plotting import pose_traces, plot_3d_points
from lac.util import rmse

In [29]:
# Wheel centers (x,y,z) in the body frame
wheel_centers = np.array(
    [
        [0.222, 0.203, 0.041],  # Front left
        [0.222, -0.203, 0.041],  # Front right
        [-0.222, 0.203, 0.041],  # Rear left
        [-0.222, 0.203, 0.041],
    ]
)  # Rear right
wheel_diameter = 0.32
rover_height = 0.134  # Z offset from robot center to lowest part of wheels

# Wheel contact points (x,y,z) in the body frame
wheel_rig = np.array(
    [
        [0.222, 0.203, -0.134],
        [0.222, -0.203, -0.134],
        [-0.222, 0.203, -0.134],
        [-0.222, 0.203, -0.134],
    ]
)

# Concatenate 1's as the last row to wheel_rig.T
wheel_rig_coords = np.concatenate((wheel_rig.T, np.ones((1, 4))), axis=0)

In [30]:
data_path = "../../output/data_collection_1"
json_data = json.load(open(f"{data_path}/data_log.json"))

poses = []
contact_points = []

for frame in json_data["frames"]:
    poses.append(np.array(frame["pose"]))
    points = np.array(frame["pose"]) @ wheel_rig_coords
    points = points[:3, :].T
    contact_points.append(points)

In [ ]:
fig = go.Figure(
    data=[
        go.Surface(
            x=heightmap_points[:, :, 0], y=heightmap_points[:, :, 1], z=heightmap_points[:, :, 2]
        )
    ]
    + pose_traces(poses[::10])
)
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [32]:
# Stack the contact points into a single array
contact_points_array = np.vstack(contact_points)

In [ ]:
fig = go.Figure(
    data=[
        go.Surface(
            x=heightmap_points[:, :, 0], y=heightmap_points[:, :, 1], z=heightmap_points[:, :, 2]
        )
    ]
)
fig = plot_3d_points(contact_points_array, fig=fig, color="blue", markersize=3)
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [36]:
fig.write_html("wheel_contact.html")

In [ ]:
contact_points_array

In [ ]:
cell_range = np.linspace(-13.5, 13.5, 181)


In [12]:
class HeightMap:
    def __init__(self, heightmap_path):
        self.heightmap = np.load(heightmap_path, allow_pickle=True)
        self.cell_range = np.linspace(-13.5, 13.5, 181)

    def get_height(self, x, y):
        x_idx = np.argmin(np.abs(self.cell_range - x))
        y_idx = np.argmin(np.abs(self.cell_range - y))
        return self.heightmap[x_idx, y_idx, 2]

    def get_heights(self, points):
        return np.array([self.get_height(x, y) for x, y in points])

In [ ]:
heightmap_points

In [23]:
heightmap = HeightMap(heightmap_path)

In [ ]:
heightmap.get_height(-13.4, -13.4)

In [ ]:
rmse(contact_points_array[:, 2], heightmap.get_heights(contact_points_array[:, :2]))